# Compute $Q^{-1}$ of 4 meter rock specimen 

To analyze the effect of attenuation on the source time function pulse, we compute the frequency-dependent $Q^{-1}(\omega)$.

2024.07.17 Kurama Okubo

- 2024.07.22 update to count how many pairs we removed due to $|B|>1$.
- 2025.03.16 update for master plot

In [ ]:
import os
import obspy
from obspy import read, Stream, Trace
from scipy import signal
import scipy.io as sio
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
%matplotlib inline
import glob
from glob import glob
import numpy as np
import mpmath as mp
import pandas as pd
import datetime
from datetime import timedelta
from tqdm import tqdm
import warnings

from matplotlib import gridspec

from scipy import interpolate
from scipy.optimize import curve_fit  
import matplotlib as mpl
import pickle

import seaborn as sns 
from scipy.interpolate import CubicSpline


plt.rcParams["font.family"] = 'Arial'
# plt.rcParams["font.sans-serif"] = "DejaVu Sans, Arial, Helvetica, Lucida Grande, Verdana, Geneva, Lucid, Avant Garde, sans-serif"
plt.rcParams["font.size"] = 10
plt.rcParams["xtick.direction"] = "in"
plt.rcParams["xtick.major.size"] = 4.75
plt.rcParams["xtick.major.width"] = 0.75
plt.rcParams["xtick.minor.size"] = 3
plt.rcParams["xtick.minor.width"] = 0.4
plt.rcParams["xtick.minor.visible"] = True

plt.rcParams["ytick.direction"] = "in"
plt.rcParams["ytick.major.size"] = 4.75
plt.rcParams["ytick.major.width"] = 0.75
plt.rcParams["ytick.minor.size"] = 3
plt.rcParams["ytick.minor.width"] = 0.4
plt.rcParams["ytick.minor.visible"] = True

plt.rcParams["savefig.transparent"] = True
plt.rcParams['axes.linewidth'] = 0.75

from obspy.core.utcdatetime import UTCDateTime  
os.environ['TZ'] = 'GMT' # change time zone to avoid confusion in unix_tvec conversion
UTCDateTime.DEFAULT_PRECISION = 8

# Method to compute $Q^{-1}(\omega)$

$$ y(\omega) = s(\omega)G(\omega; \mathbf{x}) \exp{\left( - \dfrac{\omega t}{2 Q(\omega)} \right) }  $$

Thus, 

$$ \log \left| \dfrac{y(\omega)}{s(\omega)G(\omega; \mathbf{x})} \right| = - \dfrac{\omega t}{2 Q(\omega)}, $$

where $t$ is the travel time, and for P wave pulse we assume $t = r/v_p$, $r=|\mathbf{x}|$. The Green's function is numerically computed for the relative location from source to the receiver at $\mathbf{x}$.

Therefore,
$$  Q(\omega) = - \dfrac{\omega t}{2 \ln \left| \dfrac{y(\omega)}{s(\omega)G(\omega; \mathbf{x})} \right|} $$

$$  Q^{-1}(f) = - \dfrac{1}{\pi f t}  \ln \left| \dfrac{y(f)}{s(f)G(f; \mathbf{x})} \right| $$

We plot the right side for each balldrop-AE sensor pair, and stack them to obtain the averaged $Q^{-1}(\omega)$.

We denote $\dfrac{y(\omega)}{s(\omega)G(\omega; \mathbf{x})}  = B(\omega)$, which is computed in the previous notebook `08_plot_surfaceeffect_result_Case2.ipynb`.


## remove the negative $Q^{-1}(\omega)$ due to the noise of spectrum 
Due to the noise of spectra, sometimes $|B(\omega)|$ becomes greater than 1, which is not appropriate as the observation is amplified by $|B(\omega)|$. Therefore, we use the $Q^{-1}(\omega)$ only with $|B(\omega)|<1$ to obtain the stacked attenuation factor.

In [ ]:
figdir = "../figure/09_Qinv/"
if not os.path.exists(figdir):
    os.makedirs(figdir)

# Compute the attenuation factor

In [ ]:
# load B(omega)
with open('../data/Bomega_all.pickle', mode='rb') as fi:
    Bomega_all = pickle.load(fi)


In [ ]:
Bomega_keys = list(Bomega_all)

In [ ]:
Bomega_keys

datacases=[]
for key in Bomega_all:
    if ("BD" in key) & (key[:13] not in datacases):
        datacases.append(key[:13])

In [ ]:
Ncases = len(datacases)
print(f"Ncases: {Ncases}")
vp = 6200 #[m/s]

NFFT = Bomega_all["NFFT"]
dt = Bomega_all["dt"]
freq = np.fft.rfftfreq(NFFT, dt)

Qinv_all = np.zeros((Ncases, len(freq)-1), float) # we avoid f=0

In [ ]:
np.log(np.e)

In [ ]:
VR_threshold = 0.95
const_Bover1 = np.nan #0.96 #np.nan #0.99 #np.nan # skip the value when B(omega)>1

count_pairs_VRthresh = 0
dist_pairs_VRthresh = []

for i, dataindex in enumerate(datacases):
    # i = 0
    # dataindex = datacases[i]

    dist = Bomega_all[f"{dataindex}_dist"]
    VR = Bomega_all[f"{dataindex}_VR"]
    if VR<VR_threshold:
        continue
    else:
        count_pairs_VRthresh += 1
        dist_pairs_VRthresh.append(dist)

    tt = dist*1e-3/vp # [s]
#     print(tt*1e6)

    Battenu = Bomega_all[f"{dataindex}_Battenu"]
    Battenu_smoothed = Bomega_all[f"{dataindex}_Battenu_smoothed"]
    
    # remove the inconsistent data with B(omega)
    
    # compute to align a given resp
    # const_Bover1_byQ = np.exp(5e-3 * -np.pi * 2e5 * tt)
    
    # Battenu[np.abs(Battenu)>1] = const_Bover1 # const_Bover1_byQ  #const_Bover1
    Battenu_smoothed[np.abs(Battenu_smoothed)>1] =  const_Bover1 #const_Bover1_byQ # const_Bover1
    
#     Qinv_all[i, :] = -(1/(np.pi*freq[1:]*tt)) * np.log(np.abs(Battenu[1:]))
    Qinv_all[i, :] = -(1/(np.pi *freq[1:]*tt)) * np.log(np.abs(Battenu_smoothed[1:]))
#     print(np.sign(np.log(np.abs(Battenu[1:]))))

In [ ]:
print(f"Number of pairs: {count_pairs_VRthresh}/{len(datacases)}")

print(f"Distance of pairs: {np.mean(dist_pairs_VRthresh)} ± {np.std(dist_pairs_VRthresh)}mm")


In [ ]:
Battenu_smoothed

In [ ]:
# count the non-nan values
print(Qinv_all.shape)
np.count_nonzero(~np.isnan(Qinv_all), axis=0)


In [ ]:
# plot figure for debugging
fig, ax = plt.subplots(1, 1, figsize=(8, 6))

for i, dataindex in enumerate(datacases):
    
    dist = Bomega_all[f"{dataindex}_dist"]
#     if dist > 90:
#         continue
    
    ax.loglog(freq[1:]*1e-6, Qinv_all[i, :], "o",)# c="gray", ms=3) # low VR cases are not plotted in log
#     ax.plot(freq[1:]*1e-6, Qinv_all[i, :], "o",)# c="gray", ms=3)
    
# plot P waveform
ax.set_xlim([0.06,1.])
# ax.set_ylim([1e-4, 1])
# ax.set_ylim([-0.1, 0.1])

ax.grid(True, c=np.array([230, 230, 230])/255, lw=0.25, which="both",)

# plot constant B slope f^{-1}

constB_k = 1e3
constB = constB_k*freq[1:]**(-1)
ax.plot(freq[1:]*1e-6, constB, "k--")

# ax.legend(loc=0)

ax.set_xlabel("Frequency [MHz]")
ax.set_ylabel("Attenuation, Q$^{-1}$")

# Compute stacked $Q^{-1}$

We compute the statistics of the data to obtain the quartile of $Q^{-1}$ and get the median curve, which is used to compute the attenuation factor.

In [ ]:
# Store the data to dataframe
# df_Qinv = pd.DataFrame(data=Qinv_all, index=datacases, columns=freq[1:])

df_Qinv = pd.DataFrame(columns=["dataindex", "freq", "Qinv", "distance[mm]"])

for i, dataindex in enumerate(datacases):
    # i = 0
    # dataindex = datacases[i]

    dist = Bomega_all[f"{dataindex}_dist"]
    VR = Bomega_all[f"{dataindex}_VR"]
    
    if VR<VR_threshold:
        continue
        
    tt = dist*1e-3/vp # [s]
    #     print(tt*1e6)

    Battenu = Bomega_all[f"{dataindex}_Battenu"]
    Battenu_smoothed = Bomega_all[f"{dataindex}_Battenu_smoothed"]

    # remove the inconsistent data with B(omega)<1
    Battenu[np.abs(Battenu)>1] = const_Bover1
    Battenu_smoothed[np.abs(Battenu_smoothed)>1] = const_Bover1

    #     Qinv_all[i, :] = -(1/(np.pi*freq[1:]*tt)) * np.log(np.abs(Battenu[1:]))
    Qinv_tmp= -(1/(np.pi*freq[1:]*tt)) * np.log(np.abs(Battenu_smoothed[1:]))
    #     print(np.sign(np.log(np.abs(Battenu[1:]))))

    data = {"freq":freq[1:]/1e6, "Qinv":Qinv_tmp}

    df_tmp = pd.DataFrame.from_dict(data)
    df_tmp.loc[:, "dataindex"] = dataindex
    df_tmp.loc[:, "distance[mm]"] = dist
    df_tmp.loc[:, "VR"] = VR

    if not df_Qinv.empty:
        df_Qinv = pd.concat([df_Qinv, df_tmp], ignore_index=True)
    else:
        df_Qinv = df_tmp

In [ ]:
Npair_valid_all = np.zeros((len(freq[1:]), 2)) # count the number where |B(ω)| <= 1
for i, f in enumerate(freq[1:]/1e6):
    df_tmp = df_Qinv[df_Qinv["freq"]==f]
    Nnan = df_tmp["Qinv"].isna().sum()
    Npair_valid = count_pairs_VRthresh-Nnan
    Npair_valid_all[i, :] = [f, Npair_valid]
#     print(f, Nnan, )

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
# ax.bar(x[:-1], y, width=np.diff(x), log=True,ec="k", align="edge")
ax.plot(Npair_valid_all[:, 0], Npair_valid_all[:, 1], "o")
ax.set_xscale('log')

In [ ]:
print(f"Maximum removal at {Npair_valid_all[2, 0]}MHz is {count_pairs_VRthresh -  Npair_valid_all[2, 1]}")

In [ ]:
# Npair_valid_all

In [ ]:
Npair_valid_all.shape

In [ ]:
# Number of nan due to B(omega)>1
np.sum(np.isnan(df_Qinv["Qinv"]))

In [ ]:
len(df_Qinv)/128

In [ ]:
freq

In [ ]:
df_Qinv

In [ ]:
df_Qinv.Qinv.min()

In [ ]:
df_Qinv["distance[mm]"].max()

In [ ]:
# compute quartiles of Qinv

# .quantile([0.25, 0.5, 0.75])
qinv_25 = np.zeros(len(freq[1:]))
qinv_50 = np.zeros(len(freq[1:]))
qinv_75 = np.zeros(len(freq[1:]))

for i, f in enumerate(freq[1:]):
    [qinv_25[i], qinv_50[i], qinv_75[i]] = df_Qinv[df_Qinv["freq"] == f/1e6]["Qinv"].quantile([0.25, 0.5, 0.75])

In [ ]:
data = {"freq": freq[1:]/1e6, "Qinv_25": qinv_25, "Qinv_50": qinv_50, "Qinv_75": qinv_75}
df_Qinv_quantile = pd.DataFrame.from_dict(data)
df_Qinv_quantile

In [ ]:
df_Qinv[df_Qinv["freq"] == f/1e6]["Qinv"]

In [ ]:
df_Qinv["freq"] - f

In [ ]:
df_Qinv["freq"] - f

In [ ]:
lc = sns.color_palette("colorblind")
lc

In [ ]:
np.random.seed(20240718)
fig, ax = plt.subplots(1, 1, figsize=(4.7, 4.2))

# ax.grid(True)
# ax.set_axisbelow(True)

js = 4

sns.stripplot(x="freq", y="Qinv", data=df_Qinv, jitter=False, native_scale=True, log_scale=True, color="k", size=js,
              zorder=1, legend=None, ax=ax, alpha=0.1)

sns.boxplot(x="freq", y="Qinv", data=df_Qinv, showfliers=False,  ax=ax, log_scale=True, native_scale=True, width=6, 
            zorder=2, linewidth=.75, linecolor="k", fill=True, color=lc[1])


# ax.plot(df_Qinv_quantile.freq, df_Qinv_quantile.Qinv_25, "-", c="b", lw=0.75)
ax.plot(df_Qinv_quantile.freq, df_Qinv_quantile.Qinv_50, "k--", lw=1.0, zorder=4)
# ax.plot(df_Qinv_quantile.freq, df_Qinv_quantile.Qinv_75, "-", c="r", lw=0.75)

for patch in ax.patches:
    r, g, b, a = patch.get_facecolor()
    patch.set_facecolor((r, g, b, 1.0))

ax.set_xlabel("Frequency [MHz]")
ax.set_ylabel("Attenuation, $Q_p^{-1}$")

# ax.set_xscale('log')
# ax.set_yscale('log')

# reference curve of f^{-1}
constB_k = 7e3
constB = constB_k*freq[1:]**(-1)
# ax.plot(freq[1:]*1e-6, constB, "k--")

ax.set_xlim([0.06,1.])
# ax.set_xlim([1e-3 ,1.])
ax.set_ylim([1e-4, 1])

ax.xaxis.set_major_formatter(mticker.FormatStrFormatter('%.1f'))
ax.yaxis.set_major_formatter(mticker.FormatStrFormatter('%.1g'))


ax.grid(True, c=np.array([230, 230, 230])/255, lw=0.25, which="both",)
ax.set_axisbelow(True)
plt.tight_layout()

plt.savefig(figdir+"/Qinv_stats.png", format="png", dpi=70, bbox_inches="tight")
plt.savefig(figdir+"/Qinv_stats.pdf", format="pdf", bbox_inches="tight") # keep transparency


In [ ]:
df_Qinv_quantile.to_csv("../data/df_Qinv_quantile.csv")

## Search min and max Q

In [ ]:
# bounds the frequency range to evaluate the min and max Q
inds_freqrange = np.where((0.1 < df_Qinv_quantile["freq"]) & (df_Qinv_quantile["freq"] < 1.0))[0]

In [ ]:
Qinv_argmax = df_Qinv_quantile["Qinv_50"][inds_freqrange].idxmax()
Q_min = 1/df_Qinv_quantile["Qinv_50"][inds_freqrange].max() # inverse of Qinv
Q_min_freq = df_Qinv_quantile["freq"].loc[Qinv_argmax]

Qinv_argmin = df_Qinv_quantile["Qinv_50"][inds_freqrange].idxmin()
Q_max = 1/df_Qinv_quantile["Qinv_50"][inds_freqrange].min()
Q_max_freq = df_Qinv_quantile["freq"].loc[Qinv_argmin]


In [ ]:
# Plot min and max Q
fig, ax = plt.subplots(1, 1, figsize=(6., 5))

ax.semilogx(df_Qinv_quantile["freq"], df_Qinv_quantile["Qinv_50"], "k-")

ax.set_xlabel("Frequency [MHz]")
ax.set_ylabel("Attenuation, Q$^{-1}$")

ax.plot(Q_min_freq, 1/Q_min, "bs")
ax.plot(Q_max_freq, 1/Q_max, "ro")
ax.text(Q_min_freq, 1/Q_min, f"Qmin={Q_min:.2f}  ", ha="right")
ax.text(Q_max_freq, 1/Q_max, f"Qmax={Q_max:.2f}  ", ha="right", va="top")
print(f"Qmin: {Q_min:.2f} at {Q_min_freq*1e3} kHz")
print(f"Qmax: {Q_max:.2f} at {Q_max_freq*1e3} kHz")
# ax.set_xscale('log')
# ax.set_yscale('log')

# reference curve of f^{-1}
constB_k = 7e3
constB = constB_k*freq[1:]**(-1)
# ax.plot(freq[1:]*1e-6, constB, "k--")

ax.set_xlim([0.06,1.])
# ax.set_xlim([1e-3 ,1.])
# ax.set_ylim([1e-4, 1])




## Compute $B(\omega)$

we compute $ \exp \left( {-\dfrac{\pi f t}{Q(f)}} \right) $ as the attenuation factor.

In [ ]:
Bf = np.zeros((len(freq[1:]), 3))
dist_gouge = 185.1e-3 # gouge event distance
tt = dist_gouge/vp

for i, f in enumerate(freq[1:]):
    
    Bf[i, 0] = np.exp(-np.pi*f*tt*df_Qinv_quantile[df_Qinv_quantile["freq"]==f/1e6].Qinv_25.values[0])
    Bf[i, 1] = np.exp(-np.pi*f*tt*df_Qinv_quantile[df_Qinv_quantile["freq"]==f/1e6].Qinv_50.values[0])
    Bf[i, 2] = np.exp(-np.pi*f*tt*df_Qinv_quantile[df_Qinv_quantile["freq"]==f/1e6].Qinv_75.values[0])

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6., 5))

ax.loglog(df_Qinv_quantile.freq,Bf[:,0], "b.-", zorder=4, label="low")
ax.loglog(df_Qinv_quantile.freq,Bf[:,1], "k.-", zorder=4, label="median")
ax.loglog(df_Qinv_quantile.freq,Bf[:,2], "r.-", zorder=4, label="high")

ax.set_xlabel("Frequency [MHz]")
ax.set_ylabel("$B(f)$")

# ax.set_xscale('log')
# ax.set_yscale('log')

# reference curve of f^{-1}
constB_k = 7e3
constB = constB_k*freq[1:]**(-1)
# ax.plot(freq[1:]*1e-6, constB, "k--")


ax.set_xlim([0.06,1.])
# ax.set_xlim([1e-3 ,1.])
ax.set_ylim([1e-2, 1e1])
ax.legend(loc=0)
# ax.set_ylim([-0.1, 0.1])

ax.grid(True, c=np.array([230, 230, 230])/255, lw=0.25, which="both",)


In [ ]:
20*np.log10(np.abs(Bf[:,0]))

# compute the cut-off level (-3dB)

In [ ]:
flatlevel_inds = np.where( (0.2 < freq[1:]/1e6) & (freq[1:]/1e6 < 0.3) )
cutoff_dB = np.zeros((3, 2))

freq_q = np.logspace(-1, 0, 201)

for i in range(3):
    Bf_tr_dB = 20*np.log10(np.abs(Bf[:,i]))
    flatlevel = np.mean(Bf_tr_dB[flatlevel_inds])
    print(flatlevel)
    cutoff_level = flatlevel - 3.0
    
    # interpolate the B(omega)
    Bf_tr_interp = np.interp(freq_q, df_Qinv_quantile.freq, Bf_tr_dB)
    
    cutoff_ind = np.where(Bf_tr_interp < cutoff_level)[0][0]
    cutoff_dB[i, :] = [freq_q[cutoff_ind], Bf_tr_interp[cutoff_ind]]
    

In [ ]:
cutoff_dB

In [ ]:
df_Qinv_quantile.freq

In [ ]:
cutoff_dB

In [ ]:
np.where(Bf_tr_dB < cutoff_level)[0][0]

In [ ]:
# plot in dB
fig, ax = plt.subplots(1, 1, figsize=(6.5, 5))

ax.semilogx(df_Qinv_quantile.freq, 20*np.log10(np.abs(Bf[:,0])), "b-", zorder=4, label="low attenuation")
ax.semilogx(df_Qinv_quantile.freq, 20*np.log10(np.abs(Bf[:,1])), "k-", zorder=4, label="median attenuation")
ax.semilogx(df_Qinv_quantile.freq, 20*np.log10(np.abs(Bf[:,2])), "r-", zorder=4, label="high attenuation")

ax.semilogx(cutoff_dB[0, 0], cutoff_dB[0, 1], "bs")
ax.semilogx(cutoff_dB[1, 0], cutoff_dB[1, 1], "ks")
ax.semilogx(cutoff_dB[2, 0], cutoff_dB[2, 1], "rs")

ax.set_xlabel("Frequency [MHz]")
ax.set_ylabel("$|B(\\omega)| [dB]$")

# ax.set_xscale('log')
# ax.set_yscale('log')

# reference curve of f^{-1}
constB_k = 7e3
constB = constB_k*freq[1:]**(-1)
# ax.plot(freq[1:]*1e-6, constB, "k--")


ax.set_xlim([0.06,1.])
# ax.set_xlim([1e-3 ,1.])
ax.set_ylim([-30, 10])
ax.legend(loc=0)
# ax.set_ylim([-0.1, 0.1])

ax.grid(True, c=np.array([230, 230, 230])/255, lw=0.25, which="both",)

plt.tight_layout()

plt.savefig(figdir+"/Bomega_dB.png", format="png", dpi=70, bbox_inches="tight")
# plt.savefig(figdir+"/Bomega_dB.pdf", format="eps", bbox_inches="tight")


In [ ]:
cutoff_dB

### Impulse response with the $Q^{-1}$ model

In [ ]:
def get_Qinv(freq, fq, Qinv):
    # interpolate the Q from the Qinv data
    cs = CubicSpline(fq, Qinv)
    
    Qinv_interp = np.zeros(len(freq))
    for i, ff in enumerate(freq):
        if ff<fq[0]:
            Qinv_interp[i] = Qinv[0] # extrapolate the minimum frequency Qinv
        elif ff>fq[-1]:
            Qinv_interp[i] = Qinv[-1] # extrapolate the maximum frequency Qinv
        else:
            Qinv_interp[i] = cs(ff)  
                
    return Qinv_interp

In [ ]:
Npos = 301 
fmin = 0#df_Qinv_quantile.freq.values[0]*1e6
fmax = 20e6 #df_Qinv_quantile.freq.values[-1]*1e6
freqsyn = np.linspace(0, fmax, Npos)
delta_f_syn = freqsyn[1]-freqsyn[0]
delta_f_syn


In [ ]:
Qinvsyn_25 = get_Qinv(freqsyn, df_Qinv_quantile.freq.values*1e6, df_Qinv_quantile["Qinv_25"].values)
Qinvsyn_50 = get_Qinv(freqsyn, df_Qinv_quantile.freq.values*1e6, df_Qinv_quantile["Qinv_50"].values)
Qinvsyn_75 = get_Qinv(freqsyn, df_Qinv_quantile.freq.values*1e6, df_Qinv_quantile["Qinv_75"].values)

In [ ]:
# check the interpolation
fig, ax = plt.subplots(1, 1, figsize=(6.5, 5))


ax.plot(df_Qinv_quantile.freq, df_Qinv_quantile.Qinv_25, "k--", zorder=2)
ax.plot(df_Qinv_quantile.freq, df_Qinv_quantile.Qinv_50, "k--", zorder=2)
ax.plot(df_Qinv_quantile.freq, df_Qinv_quantile.Qinv_75, "k--", zorder=2)

ax.plot(freqsyn/1e6, Qinvsyn_25, "g-", zorder=1)
ax.plot(freqsyn/1e6, Qinvsyn_50, "b-", zorder=1)
ax.plot(freqsyn/1e6, Qinvsyn_75, "r-", zorder=1)


ax.set_xscale('log')
ax.set_yscale('log')

ax.set_xlabel("Frequency [MHz]")
ax.set_ylabel("Attenuation, Q$^{-1}$")

# ax.set_xscale('log')
# ax.set_yscale('log')

# reference curve of f^{-1}
constB_k = 7e3
constB = constB_k*freq[1:]**(-1)
# ax.plot(freq[1:]*1e-6, constB, "k--")


ax.set_xlim([1e-3,5.])
# ax.set_xlim([1e-3 ,1.])
ax.set_ylim([6e-4, 1])

# ax.set_ylim([-0.1, 0.1])

ax.grid(True, c=np.array([230, 230, 230])/255, lw=0.25, which="both",)


In [ ]:
# compute attenuation factor
Bfsyn = np.zeros((len(freqsyn), 3))
dist_gouge = 185.1e-3 # gouge event distance
tt = dist_gouge/vp

for i, ff in enumerate(freqsyn):
    
    Bfsyn[i, 0] = np.exp(-np.pi*ff*tt*Qinvsyn_25[i])
    Bfsyn[i, 1] = np.exp(-np.pi*ff*tt*Qinvsyn_50[i])
    Bfsyn[i, 2] = np.exp(-np.pi*ff*tt*Qinvsyn_75[i])


In [ ]:
# Nrfft = 2*10

y_25 = np.fft.irfft(Bfsyn[:,0])
y_50 = np.fft.irfft(Bfsyn[:,1])
y_75 = np.fft.irfft(Bfsyn[:,2])


Ntrace = 2*(Npos-1)
assert len(y_25) == Ntrace

dt = 1/(Ntrace*delta_f_syn)
tvec = np.arange(0, Ntrace, step=1)*dt

# nyquist is 5MHz.

In [ ]:
dt

In [ ]:
plt.plot(tvec*1e6, y_25/np.max(y_25), ".b-")
plt.plot(tvec*1e6, y_50/np.max(y_50), ".k-")
plt.plot(tvec*1e6, y_75/np.max(y_75), ".g-")

# plt.plot(tvec*1e6, y_25[::-1], "xb")
# plt.plot(tvec*1e6, y_50[::-1], "xk")
# plt.plot(tvec*1e6, y_75[::-1], "xg")

plt.xlim(0, 4)
plt.xlabel("Time [μs]")

The figure above is the impulse response of the Qmodel. We further analyze this with the Futterman function.


In [ ]:
Qp = 50
tt_lab = 200e-3/6200
tt_lin2016 = 14e3/5000
tstar_lab = tt_lab/Qp
tstar_lin2016 = tt_lin2016/Qp

In [ ]:
tstar_lab

In [ ]:
tstar_lin2016

In [ ]:
tstar_lab/tstar_lin2016 # this indicates the scaling of Futterman function pulse width

The pulse width of Fig. 10b is ~0.2s. $0.2*1e-5$ = 2μs, which is comparable to the figure above. Thus, the scaling is reasonable.